In [3]:
import sys
sys.path.append("../../")
from rl.dynamic_programming import V, S, A
from rl import dynamic_programming

In [4]:
from rl import markov_process, markov_decision_process

In [5]:
import operator
from typing import Mapping, Iterator, TypeVar, Tuple, Dict, Iterable

import numpy as np

from rl.distribution import Categorical, Choose
from rl.iterate import converged, iterate
from rl.markov_process import NonTerminal, State
from rl.markov_decision_process import (FiniteMarkovDecisionProcess,
                                        FiniteMarkovRewardProcess)
from rl.policy import FinitePolicy, FiniteDeterministicPolicy

In [6]:
from rl.midterm_2022.priority_q import  PriorityQueue
from rl.midterm_2022 import grid_maze

In [7]:
import importlib

In [8]:
from time import time

# Problem 1

In this problem we will implement the gaps-based value iteration algorithm mentioned in class.

The gaps-based iteration algorithm proceeds as follows

1. Initialize the value function to zero for all states: $v[s] = 0\ \forall s \in \mathcal{N}$
2. Calculate the gaps for each state: $g[s] = |v[s] - \max_a \mathcal{R}(s,a) + \sum_{s'} \mathcal{P}(s,a,s') \cdot v(s')|$
3. While there is some gap that exceeds a threshold
 - Select the state with the  largest gap: $s_{max} = \arg\max_{s \in \mathcal{N}} g[s]$
 - Update the value function for $s_{max}$: $v[s_{max}] = \max_a \mathcal{R}(s_{max},a) + \sum_{s'}\mathcal{P}(s_{max},a,s') \cdot v(s')$
 -  Update the gap for $s_{max}$: $g[s_{max}] = 0$
4. Return v

We will test your implementation on a grid maze MDP. We have defined this class in "grid_maze.py", you should  briefly familiarize yourself with that code. In particular pay attention to the difference in reward functions for the two classes "GridMazeMDP_Dense" and "GridMazeMDP_Sparse"

Here is how you can use the classes:

In [61]:
underlying_maze = grid_maze.Maze(2, 3)
maze_mdp = grid_maze.GridMazeMDP_Sparse(underlying_maze, 0, 0)

you can visualize the maze if you wish

In [62]:
print(maze_mdp)

----
|*  |
| +-+
|   |
| + +
| | |
|-+-+


you can also visualize a policy on the mdp

In [ ]:
v2_res = dynamic_programming.value_iteration_result(maze_mdp, 0.9)
print(maze_mdp.print_policy(v2_res[1]))

You will need to make use of the PriorityQueue class in your implementation. A PriorityQueue is an ordered queue which supports the following operations
1. isEmpty(self): check if the queue is empty   
2. contains(self, element): check if the queue contains an element
3. peek(self): peek at the highest priority element in the queue    
4. pop(self): remove and return the highest priority element in the queue    
5. insert(self, element, priority): insert an element into the queue with given priority
6. update(self, element, new_priority): update the priority of an element in the queue
7. delete(self, element): delete an element from the queue

Below are some examples of using the queue

In [70]:
q: PriorityQueue = PriorityQueue()
# print(q.isEmpty(), ':', "the queue is empty")
q.insert("a", 0.4)
# print(q.isEmpty(), ':',  "the queue is not empty")
# print(q.contains("a"), ':',  "the queue contains a")
# print(q.contains("b"), ':',  "the queue does not contain a")
# print(q.peek(), ':',  "a is the first element in the queue")
q.insert("b", -0.3)
# print(q.contains("b"), ':',  "the queue now contains b")
# print(q.peek(), ':',  "b is now at the front of the queue")
# x = q.pop()
# print(x, ':',  "we removed b from the queue")
# print(q.isEmpty(), ':',  "the queue still nonempty")
# print(q.contains("a"), ':',  "the queue still contains a")
# print(q.contains("b"), ':',  "the queue does not contain b anymore")
# print(q.peek(), ':',  "a is at the front of the queue")
q.insert("c", 0.56)
# print(q.peek(), ':',  "a is still at the front of the queue")
print(q)
q.update("a", -0.3)
q.update("b", -0.4)
print(q)
# print(q.peek(), ':',  "after updating a is no longer at the front of the queue")

(-0.3, 'b') (0.4, 'a') (0.56, 'c')
(-0.4, 'b') (0.56, 'c') (-0.3, 'a')


### Part 1

In [99]:
from collections import defaultdict
def invert_transition_map(mdp: markov_decision_process.FiniteMarkovDecisionProcess[S, A]) -> Mapping[S, Iterable[S]]:
        inverted_mapping = {}

        for curr_state, act_dict in mdp.mapping.items():
                for _, sr_dict in act_dict.items():
                        for (next_state, _), _ in sr_dict.table().items():
                                if next_state not in inverted_mapping:
                                        inverted_mapping[next_state] = [curr_state]
                                else:
                                        inverted_mapping[next_state].append(curr_state)

        return inverted_mapping

### Part 2

In [109]:
def gaps_value_iteration(
    mdp: markov_decision_process.FiniteMarkovDecisionProcess[S, A],
    gamma: float, 
    gaps: PriorityQueue) -> Iterator[V[S]]:
    '''
    Calculate the value function (V*) of the given MDP by applying the
    update function repeatedly until the values converge.

    '''
    dependency_map = invert_transition_map(mdp)
    v_0: V[S] = {s: 0.0 for s in mdp.non_terminal_states}
    
        
    def update(v: V[S]) -> V[S]:
        '''
        YOUR CODE HERE
        perform a single update to v for the state with the largest gap
        update the gaps for any dependent states
        '''
        s_max : S = gaps.pop()
        v[s_max] = max(mdp.mapping[s_max][a].expectation(lambda sr : sr[1] + gamma * dynamic_programming.extended_vf(v, sr[0])) for a in mdp.actions(s_max))

        for s in dependency_map:
            if isinstance(s, NonTerminal) and gaps.contains(s):
                max_val = max(mdp.mapping[s][a].expectation(lambda sr : sr[1] + gamma * dynamic_programming.extended_vf(v, sr[0])) for a in mdp.actions(s))
                gaps.update(s, - abs(v[s] - max_val))

        return v

    
    return iterate(update, v_0)


def gaps_value_iteration_result(
    mdp: FiniteMarkovDecisionProcess[S, A],
    gamma: float
) -> Tuple[V[S], FiniteDeterministicPolicy[S, A]]:
    
    gaps = PriorityQueue()
    
    '''
    YOUR CODE HERE
    instantiate the value function and populate the gaps
    ''' 
    for s in mdp.non_terminal_states:
        max_val = max(mdp.mapping[s][a].expectation(lambda sr : sr[1]) for a in mdp.actions(s))
        gaps.insert(s, - max_val)
    
    def criterion(v1 : V[S], v2 : V[S], TOLERANCE : float = 1e-5) -> bool:
        '''
        YOUR CODE HERE
        implement the criterion for convergence of the value function 
        ''' 
        return max(abs(v1[s] - v2[s]) for s in v1) < TOLERANCE
        
    opt_vf: V[S] = converged(
        gaps_value_iteration(mdp, gamma, gaps),
        done=criterion 
    )
        
    opt_policy: markov_decision_process.FiniteDeterministicPolicy[S, A] = dynamic_programming.greedy_policy_from_vf(
        mdp,
        opt_vf,
        gamma
    )

    return opt_vf, opt_policy

# Do not change the code below here, just run it

## Calculating the VF for a maze with sparse rewards

In [112]:
underlying_maze = grid_maze.Maze(50, 50)
maze_mdp = grid_maze.GridMazeMDP_Sparse(underlying_maze, 0, 0)

#### printing the runtime for the calculation

In [118]:
start = time()
v1_res = gaps_value_iteration_result(maze_mdp, 0.9)
print(v1_res[1])
print(time() - start)

For State GridState(x=0, y=1): Do Action N
For State GridState(x=0, y=2): Do Action N
For State GridState(x=0, y=3): Do Action S
For State GridState(x=0, y=4): Do Action S
For State GridState(x=0, y=5): Do Action E
For State GridState(x=0, y=6): Do Action N
For State GridState(x=0, y=7): Do Action E
For State GridState(x=0, y=8): Do Action E
For State GridState(x=0, y=9): Do Action S
For State GridState(x=0, y=10): Do Action E
For State GridState(x=0, y=11): Do Action E
For State GridState(x=0, y=12): Do Action N
For State GridState(x=0, y=13): Do Action E
For State GridState(x=0, y=14): Do Action S
For State GridState(x=0, y=15): Do Action N
For State GridState(x=0, y=16): Do Action E
For State GridState(x=0, y=17): Do Action E
For State GridState(x=0, y=18): Do Action N
For State GridState(x=0, y=19): Do Action E
For State GridState(x=0, y=20): Do Action S
For State GridState(x=0, y=21): Do Action N
For State GridState(x=0, y=22): Do Action E
For State GridState(x=0, y=23): Do Action

In [119]:
start = time()
v2_res = dynamic_programming.value_iteration_result(maze_mdp, 0.9)

print(v2_res[1])
print(time() - start)

For State GridState(x=0, y=1): Do Action N
For State GridState(x=0, y=2): Do Action N
For State GridState(x=0, y=3): Do Action N
For State GridState(x=0, y=4): Do Action N
For State GridState(x=0, y=5): Do Action N
For State GridState(x=0, y=6): Do Action N
For State GridState(x=0, y=7): Do Action E
For State GridState(x=0, y=8): Do Action E
For State GridState(x=0, y=9): Do Action S
For State GridState(x=0, y=10): Do Action E
For State GridState(x=0, y=11): Do Action E
For State GridState(x=0, y=12): Do Action N
For State GridState(x=0, y=13): Do Action E
For State GridState(x=0, y=14): Do Action N
For State GridState(x=0, y=15): Do Action N
For State GridState(x=0, y=16): Do Action E
For State GridState(x=0, y=17): Do Action E
For State GridState(x=0, y=18): Do Action N
For State GridState(x=0, y=19): Do Action E
For State GridState(x=0, y=20): Do Action N
For State GridState(x=0, y=21): Do Action N
For State GridState(x=0, y=22): Do Action E
For State GridState(x=0, y=23): Do Action

#### confirming that the value functions are identical

In [115]:
assert v1_res[1] == v2_res[1]

AssertionError: 

## Calculating the VF for a maze with dense rewards

In [ ]:
maze_mdp = grid_maze.GridMazeMDP_Dense(underlying_maze, 0, 0)

#### printing the runtime for the calculation

In [ ]:
start = time()
v1_res = gaps_value_iteration_result(maze_mdp, 1)
print(time() - start)

In [ ]:
start = time()
v2_res = dynamic_programming.value_iteration_result(maze_mdp, 1)
print(time() - start)

#### confirming that the value functions are identical

In [ ]:
assert v1_res[1] == v2_res[1]